In [222]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import json
import os

In [321]:
fanfics = os.listdir('Fanfic_HTML')

fanfics.remove('.DS_Store')

for index, item in enumerate(fanfics):
    fanfics[index] = 'Fanfic_HTML/' + item

In [247]:
def extractStoryRating(classTags):
    #code to extract story rating
    i = 0
    for element in classTags:
        i += 1
        if i == 4:
            story_rating = str(element.find('a').string)
        elif i > 4:
            break
            
    return story_rating

def extractArchiveWarning(classTags):
    i = 0
    for element in classTags:
        i += 1
        if i == 8:
            archiveWarning = str(element.find('a').string)
        elif i > 8:
            break
            
    return archiveWarning

def extractCategory(classTags):
    #code to extract category
    i = 0
    for element in classTags:
        i += 1
        if i == 12:
            story_category = str(element.find('a').string)
        elif i > 12:
            break

    return story_category

def extractFandoms(classTags):
    #code to extract fandoms
    i = 0
    j = 0
    fandom_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Fandom:":
            j = 2 + i
        elif i == j:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += str(obj.string) + ', '
            fandom_text = fandom_text[:-2]

    return fandom_text

def extractRelationships(element):
    #code to extract relationships
    i = 0
    j = 0
    story_relationships = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Relationship:":
            j = 2 + i
        elif i == j:
            raw_relationships = element.find_all('a')
            for obj in raw_relationships:
                story_relationships += str(obj.string) + ', '
            story_relationships = story_relationships[:-2]
            
    return story_relationships

def extractCharacters(classTags):
    #code to extract characters
    i = 0
    j = 0
    characters = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Character:":
            j = 2 + i
        elif i == j:
            raw_characters = element.find_all('a')
            for obj in raw_characters:
                characters += str(obj.string) + ', '
            characters = characters[:-2]
            
    return characters

def extractAddTags(classTags):
    #code to extract tags
    i = 0
    j = 0
    additionalTags_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Additional Tags:":
            j = 2 + i
        elif i == j:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += str(obj.string) + ', '
            additionalTags_text = additionalTags_text[:-2]

    return additionalTags_text

def extractSeries(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Series:":
            j = 2 + i
        elif i == j:
            series = str(element.find('a').string)

    return series

def extractCollections(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    collections = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Collections:":
            j = 2 + i
        elif i == j:
            collections = str(element.find('a').string)

    return collections

def extractPubStats(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Stats:":
            j = 2 + i
        elif i == j:
            pub_stat = str(element.string).strip()
            story_publish_date = pub_stat[11:21]
            story_word_count = pub_stat[37:40]
    
    return story_publish_date, story_word_count

In [248]:
def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = np.nan
    story_character = np.nan
    relationship = np.nan
    addTags = np.nan
    archiveWarning = np.nan
    category = np.nan
    fandoms = np.nan
    pubDate = np.nan
    series = np.nan
    collections = np.nan
    wordCount = np.nan
    tagsClass = soup_obj.find(class_="tags")
    for element in tagsClass:
        if str(element.string) == "Rating:":
            rating = extractStoryRating(tagsClass)
            
        elif str(element.string) == "Archive Warning:":
            archiveWarning = extractArchiveWarning(tagsClass)
            
        elif str(element.string) == "Category:":
            category = extractCategory(tagsClass)
            
        elif str(element.string) == "Fandom:":
            fandom_text = extractFandoms(tagsClass)
        
        elif str(element.string) == "Relationship:":
            relationship = extractRelationships(tagsClass)
            
        elif str(element.string) == "Character:":
            story_character = extractCharacters(tagsClass)
            
        elif str(element.string) == "Additional Tags:":
            addTags = extractAddTags(tagsClass)
            
        elif str(element.string) == "Series:":
            series = extractSeries(tagsClass)
            
        elif str(element.string) == "Collections:":
            collections = extractCollections(tagsClass)
            
        elif str(element.string) == "Stats:":
            pubDate, wordCount = extractPubStats(tagsClass)
            
        i += 1
    
    local_tag_dict = {'rating' : rating,
                'archiveWarnings': archiveWarning,
                'category' : category,
                'fandom' : fandom_text,
                'relationships' : relationship,
                'character': story_character,
                'additionalTags': addTags,  
                'series': series, 
                'collections': collections,
                'pub_date' : pubDate, 
                'word_count' : wordCount
               }
        
    return local_tag_dict

In [314]:
def extractAuthor(soup_obj):
    #code to extract author and fandom info
    for info in soup_obj.title:
        author = str(info).split('-')[1]
        author = author.strip()
    return author

def extractTitle(soup_obj):
    #code to extract title
    i = 0
    for element in soup_obj.find(class_="message"):
        if i == 1:
            title = element.string
        elif i > 1:
            break
        i += 1
        
    return title
    
def extractStory(soup_obj):
    #code to extract title and story text
    i = 0
    story_text = ""
    for element in soup_obj.find(id="chapters"):
        if i == 3:
            story_ptag_text = element.find_all('p')
            for obj in story_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_text += str(obj.string) + " "
                    
        elif i > 3:
            break
            
        i += 1
        
    return story_text


def extractStoryChapters(soup_obj):
    #code to extract title and story text
    i = 0
    j = 0
    story_chapter_text = ""
    chapters = soup_obj.find(id="chapters")
    for element in soup_obj.find(id="chapters"):        
        i += 1
        if str(element.string) == "chapter content":
            j = 2 + i
        elif str(element.string) == "/chapter content":
            j = 2 + i
        elif i == j:
            userStuff = chapters.find(class_="userstuff")
            chapter_ptag_text = userStuff.find_all('p')
            for obj in chapter_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_chapter_text += str(obj.string) + " " 
                    
    return story_chapter_text

def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
    return url

def createAO3dict(filelist):
    i = 0
    ao3_dict = {}
    for element in filelist:
        soup = BeautifulSoup(open(element), "html.parser")
        url = extractStoryURL(soup)
        tag_dict = extractTags(soup)
        author = extractAuthor(soup)
        title = extractTitle(soup)
        if "chapter content" in str(soup):
            story_text = extractStoryChapters(soup)
        else:
            story_text = extractStory(soup)
        
        story_dict = {'title': title,
                      'author': author,
                      'story_text': story_text, 
                      'url': url,
                     }
        story_dict.update(tag_dict)
        
        ao3_dict[i] = story_dict
        i += 1
    return ao3_dict

In [323]:
ao3_dictionary = createAO3dict(fanfics)

Fanfic_HTML/One Too Many Accidents.html
Fanfic_HTML/The Life Plan.html
Fanfic_HTML/One Recurse.html
Fanfic_HTML/ask and ye shall receive.html
Fanfic_HTML/In Gardens.html
Fanfic_HTML/Go Down Shooting.html
Fanfic_HTML/Not quite a fix-it.html
Fanfic_HTML/The Morrigan.html
Fanfic_HTML/Cheeky September.html
Fanfic_HTML/Sun and Moon.html
Fanfic_HTML/affair with the ground.html
Fanfic_HTML/A Running Graceling Is.html
Fanfic_HTML/Five Times Katniss.html
Fanfic_HTML/say youll remember me.html
Fanfic_HTML/there cannot only be one.html
Fanfic_HTML/Katsas Guide to the.html
Fanfic_HTML/All That I Require.html
Fanfic_HTML/Made in Rings of Rain.html
Fanfic_HTML/Everything Before and.html
Fanfic_HTML/heart palpitations.html
Fanfic_HTML/Without Thinking.html
Fanfic_HTML/They Took Him.html
Fanfic_HTML/Breakfast in Bed.html
Fanfic_HTML/Birthing Pangs.html
Fanfic_HTML/Tidbits Conversations.html
Fanfic_HTML/Theyll Heal.html
Fanfic_HTML/the best thing we do.html
Fanfic_HTML/Braiding Weaving.html
Fanfic_HTML

Fanfic_HTML/theres a fire down below.html
Fanfic_HTML/Help me Find the Fic.html
Fanfic_HTML/A Thing.html
Fanfic_HTML/Everlark Post-Mockingjay.html
Fanfic_HTML/Two Days One Toasting.html
Fanfic_HTML/Cages and Sanctuaries.html
Fanfic_HTML/Words.html
Fanfic_HTML/Distractions.html
Fanfic_HTML/breathe in exhale the.html
Fanfic_HTML/The Rival.html
Fanfic_HTML/Necessitarianism.html
Fanfic_HTML/Symbol of Reform.html
Fanfic_HTML/Cold.html
Fanfic_HTML/Self Insert.html
Fanfic_HTML/Red Phone.html
Fanfic_HTML/Declarations.html
Fanfic_HTML/The Future.html
Fanfic_HTML/Various Uses for Towels-1.html
Fanfic_HTML/Insecurity.html
Fanfic_HTML/The Disappearing Brother.html
Fanfic_HTML/A sleepy morning.html
Fanfic_HTML/The Journey West.html
Fanfic_HTML/Chai Lattes and crushes.html
Fanfic_HTML/A Heart Laid To Waste.html
Fanfic_HTML/The Zogist Salute.html
Fanfic_HTML/how the hargreeves kids.html
Fanfic_HTML/legacy.html
Fanfic_HTML/Not A Bit Like Zeus.html
Fanfic_HTML/it was never you.html
Fanfic_HTML/Spiderwe

Fanfic_HTML/Dust and Moonbeams.html
Fanfic_HTML/The Plural Pronoun.html
Fanfic_HTML/Red Sky.html
Fanfic_HTML/Dont let him stab you.html
Fanfic_HTML/A Wrinkle in Time Kiss.html
Fanfic_HTML/Mistress of Wakanda.html
Fanfic_HTML/Battle.html
Fanfic_HTML/Mere Mortal.html
Fanfic_HTML/Child Prodigy.html
Fanfic_HTML/Alone Together.html
Fanfic_HTML/Second Honeymoon.html
Fanfic_HTML/Waiting.html
Fanfic_HTML/Stay At My Place If You.html
Fanfic_HTML/as all the flowers in.html
Fanfic_HTML/Even if I come back even.html
Fanfic_HTML/Emma Enchanted.html
Fanfic_HTML/A day in Wakanda.html
Fanfic_HTML/These Boundaries Crossed.html
Fanfic_HTML/The Bakers Weakness.html
Fanfic_HTML/Hed Give The Stars.html
Fanfic_HTML/Graceless Killing.html
Fanfic_HTML/a hacker a hitter and a.html
Fanfic_HTML/Call me by my name.html
Fanfic_HTML/but doth must suffer a.html
Fanfic_HTML/Lightning in your Eyes.html
Fanfic_HTML/all youre good for.html
Fanfic_HTML/Pleasure Before Business.html
Fanfic_HTML/Goodbyes.html
Fanfic_HTML/C

Fanfic_HTML/Thirty Seconds.html
Fanfic_HTML/You Oughta Know But Im.html
Fanfic_HTML/The Call.html
Fanfic_HTML/Star Gazing with Howl.html
Fanfic_HTML/The Queens Proposal.html
Fanfic_HTML/Sartorial Matters.html
Fanfic_HTML/Take My Hand The Second.html
Fanfic_HTML/Drabble for the Road -.html
Fanfic_HTML/Three.html
Fanfic_HTML/Exchange(1).html
Fanfic_HTML/Old.html
Fanfic_HTML/Red Brick River.html
Fanfic_HTML/The Internet Is Forever.html
Fanfic_HTML/Cold copy.html
Fanfic_HTML/Possibly The Fundamental.html
Fanfic_HTML/We Can Try to Fix Our.html
Fanfic_HTML/Goodnight Ben.html
Fanfic_HTML/chromesthesia.html
Fanfic_HTML/Kinkless.html
Fanfic_HTML/Book Commentary.html
Fanfic_HTML/Ford makes a Bargain.html
Fanfic_HTML/Late.html
Fanfic_HTML/Is This the Best You Can.html
Fanfic_HTML/reaching an.html
Fanfic_HTML/Po and katsa moments.html
Fanfic_HTML/Grace and Remembrance.html
Fanfic_HTML/Worth the Wait.html
Fanfic_HTML/Best Dinner Ever.html
Fanfic_HTML/how dare you speak of.html
Fanfic_HTML/Getting T

Fanfic_HTML/Save Me A Seat.html
Fanfic_HTML/Dear Father dont give me.html
Fanfic_HTML/When the Whiskey Goes In.html
Fanfic_HTML/Lessons.html
Fanfic_HTML/No Such Thing as.html
Fanfic_HTML/A Dandelion From The.html
Fanfic_HTML/In Which Howl Attracts.html
Fanfic_HTML/Enough Rope.html
Fanfic_HTML/A Good Days Work Leaving.html
Fanfic_HTML/Power Play.html
Fanfic_HTML/Downtime.html
Fanfic_HTML/Returning.html
Fanfic_HTML/Blend.html
Fanfic_HTML/Baby Its Cold Outside.html
Fanfic_HTML/Journeys End.html
Fanfic_HTML/Exchange.html
Fanfic_HTML/Peeta Mellark In it to.html
Fanfic_HTML/Keeping Promises.html
Fanfic_HTML/Interlude It grows back.html
Fanfic_HTML/Partnership.html


In [324]:
ao3_df = pd.DataFrame.from_dict(ao3_dictionary, orient="index")
ao3_df

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count
0,One Too Many Accidents,kazesuke,"“Sophiiiieee!!!” Loud, wet footsteps slammed a...",http://archiveofourown.org/works/299981,General Audiences,No Archive Warnings Apply,Gen,Howl no Ugoku Shiro | Howl's Moving Castle,NaN,"Howl Pendragon, Sophie, Markl, Calcifer (Howl ...",Genderbending,NaN,Yuletide 2011,2011-12-22,107
1,The Life Plan,Anonfandom101,Life. ( A brief albeit important prolouge )\n\...,http://archiveofourown.org/works/19198501,Teen And Up Audiences,No Archive Warnings Apply,F/F,Hitchhiker's Guide to the Galaxy - Douglas Ada...,NaN,NaN,NaN,NaN,NaN,2019-06-15,750
2,One Recurse,orphan_account,No one of them could be Cindi Mayweather. It's...,http://archiveofourown.org/works/333825,Mature,No Archive Warnings Apply,F/F,"Metropolis: The Chase Suite - Janelle Monáe, T...",NaN,"Alpha 9000s, Cindi Mayweather","Androids, Clones, Character of Color, Porn Battle",NaN,Porn Battle XIII (Lucky Thirteen),2012-02-06,889
3,"ask, and ye shall receive",Ariaste,"He feels rather daring about it, especially wh...",http://archiveofourown.org/works/19964368,General Audiences,No Archive Warnings Apply,M/M,"Good Omens (TV), Good Omens - Neil Gaiman & Te...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...",NaN,"a tiny accidental fic, cute and sweet, puns",NaN,My Favorite Ineffable Husband Fics,2019-07-24,441
4,In Gardens,Bitterblue,,http://archiveofourown.org/works/432750,General Audiences,No Archive Warnings Apply,Gen,"Bitterblue - Kristin Cashore, Seven Kingdoms T...",NaN,"Hava (Bitterblue), Bitterblue (Graceling)",NaN,NaN,NaN,2012-06-13,279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,Exchange,swiddershins,"He’d inquired about the coat’s price, carefull...",http://archiveofourown.org/works/14392434,General Audiences,No Archive Warnings Apply,Gen,"Neverwhere - Neil Gaiman, Neverwhere - All Med...",NaN,Marquis de Carabas (Neverwhere),"baby marquis, baby baby baby marquis, Characte...",NaN,NaN,2018-04-22,759
1046,Peeta Mellark: In it to Survive,SilverTonguedWriter,The games were tomorrow. I had no fall back pl...,http://archiveofourown.org/works/1000334,Mature,Choose Not To Use Archive Warnings,M/M,Hunger Games Trilogy - Suzanne Collins,"Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Cato (Hunger Games), Peeta Mellark","One Shot, Sex, Blow Jobs",NaN,NaN,2013-10-11,444
1047,Keeping Promises,fourredfruits,Everything happened so fast it seemed like the...,http://archiveofourown.org/works/18372194,General Audiences,Choose Not To Use Archive Warnings,F/M,"The Umbrella Academy (TV), The Umbrella Academ...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Number Five | The Boy (Umbrella Academy), Vany...",NaN,NaN,NaN,2019-04-06,808
1048,Interlude: It grows back,kaleigh,,http://archiveofourown.org/works/21219572,Not Rated,No Archive Warnings Apply,F/F,"Addams Family - All Media Types, Addams Family...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Wednesday Addams, Pubert Addams, Lurch (Addams...","Haircuts, Sibling Bonding",Love is a four letter word,NaN,2019-10-29,959


In [328]:
ao3_df[ao3_df["story_text"] == '']

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count
4,In Gardens,Bitterblue,,http://archiveofourown.org/works/432750,General Audiences,No Archive Warnings Apply,Gen,"Bitterblue - Kristin Cashore, Seven Kingdoms T...",NaN,"Hava (Bitterblue), Bitterblue (Graceling)",NaN,NaN,NaN,2012-06-13,279
6,Not quite a fix-it,it,,http://archiveofourown.org/works/8010814,General Audiences,Major Character Death,Other,The Giver Series - Lois Lowry,NaN,"Jonas (The Giver), Gabe (The Giver), Lily (The...",NaN,NaN,NaN,2016-09-11,437
12,Five Times Katniss Everdeen Kissed Peeta Mella...,gavorn,,http://archiveofourown.org/works/1086052,Teen And Up Audiences,No Archive Warnings Apply,F/M,"Hunger Games Trilogy - Suzanne Collins, Hunger...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Katniss Everdeen, Clove (Hunger Games), Peeta ...","Universe Alteration, That's right I went there...",NaN,NaN,2013-12-17,969
14,there can(not) only be one,TheTartWitch,,http://archiveofourown.org/works/15776958,Teen And Up Audiences,Major Character Death,Gen,Fire - Kristin Cashore,"Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Fire - Character, Cansrel, OMC, Hanna, Blotchy...","Cansrel dies later, Fire has a twin, mindmeld,...",NaN,NaN,2018-08-23,350
32,New Beginnings AU,lilC0le,,http://archiveofourown.org/works/15513993,Mature,No Archive Warnings Apply,F/M,"Marvel, Black Panther - Fandom","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","black OC: aja parker, Erik Killmonger, T’Chall...","AU, College AU, Alternate Universe - College/U...",NaN,NaN,2018-07-31,ers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,You Were Just Too Kind and I Was Too Young to ...,Huntress8611,,http://archiveofourown.org/works/18393527,General Audiences,Choose Not To Use Archive Warnings,Gen,"The Umbrella Academy, The Umbrella Academy (TV)","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Original Characters, Original Male Character(s...","Hurt/Comfort, Emotional Hurt/Comfort, Fear, Vi...","Who Was Wrong and Who Was Right, It Didn't Mat...",NaN,2019-04-08,751
964,In the Courtyard,Bitterblue,,http://archiveofourown.org/works/438101,Mature,No Archive Warnings Apply,F/M,"Bitterblue - Kristin Cashore, Seven Kingdoms T...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Katsa (Graceling), Po (Graceling), Bitterblue ...",NaN,NaN,NaN,2012-06-19,771
980,Book Commentary: The Book Thief,mlmcg12,,http://archiveofourown.org/works/28968042,Teen And Up Audiences,Graphic Depictions Of Violence,Gen,The Book Thief - Markus Zusak,NaN,Death (The Book Thief),"Spoilers, Cross-Posted on WordPress",Book Blog Transcripts,NaN,2019-12-10,ers
981,Shiraz,Lbug84,,http://archiveofourown.org/works/2225220,Teen And Up Audiences,Choose Not To Use Archive Warnings,F/F,"Hunger Games Trilogy - Suzanne Collins, Hunger...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Annie Cresta, Finnick Odair, Katniss Everdeen,...","Everlarkesta, Polyamory, Foursome, Threesome",The Glühwein Series,Prompts in Panem - August 2014,2014-08-30,686


In [329]:
ao3_df[ao3_df["word_count"] == 'ers']

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count
18,Everything Before and After You,n0luv,"It’s funny that this happened to him. Him, o...",http://archiveofourown.org/works/29665824,Teen And Up Audiences,Graphic Depictions Of Violence,Gen,"The Umbrella Academy (TV), The Umbrella Academ...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Luther Hargreeves, Five Hargreeves, Vanya Harg...","Angst, Hurt No Comfort, Time Travel, Time Loop...",NaN,NaN,2021-03-16,ers
32,New Beginnings AU,lilC0le,,http://archiveofourown.org/works/15513993,Mature,No Archive Warnings Apply,F/M,"Marvel, Black Panther - Fandom","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","black OC: aja parker, Erik Killmonger, T’Chall...","AU, College AU, Alternate Universe - College/U...",NaN,NaN,2018-07-31,ers
51,"smile, though your heart is breaking",ange1teeth,"the smell of cheap, cinnamon whiskey is a fami...",http://archiveofourown.org/works/18233057,Not Rated,No Archive Warnings Apply,Other,"The Umbrella Academy (TV), The Umbrella Academ...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Klaus Hargreeves, Number Five | The Boy (Umbre...","No Incest, Don't be hecking nasty, no smut eit...",NaN,NaN,2019-11-21,ers
66,He's the Angel of Death,Huntress8611,Oliver heard a loud scream over the sound of g...,http://archiveofourown.org/works/18322775,Teen And Up Audiences,Graphic Depictions Of Violence,Gen,"The Umbrella Academy, The Umbrella Academy (TV)","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Klaus Hargreeves, Dave Katz, Original Characte...","Original Character Death(s), Guns, Vietnam War...","Who Was Wrong and Who Was Right, It Didn't Mat...",I’m love,2019-04-02,ers
72,send me on my way,kaminagi,,http://archiveofourown.org/works/13538607,General Audiences,Choose Not To Use Archive Warnings,Gen,Matilda - Roald Dahl,"Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Matilda Wormwood, Jennifer Honey","3 Sentence Ficathon, Libraries",NaN,NaN,2018-02-09,ers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,My Favorite Mistake,titania522,Fic has been removed to make into an original ...,http://archiveofourown.org/works/5033671,Explicit,Choose Not To Use Archive Warnings,Gen,"The Hunger Games (Movies), Hunger Games Trilog...","Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...","Katniss Everdeen, Peeta Mellark, Haymitch Aber...","tw: infidelity, everlark, everlark fanfiction,...",NaN,Prompts in Panem - Farewell Tour 2015,2015-10-19,ers
980,Book Commentary: The Book Thief,mlmcg12,,http://archiveofourown.org/works/28968042,Teen And Up Audiences,Graphic Depictions Of Violence,Gen,The Book Thief - Markus Zusak,NaN,Death (The Book Thief),"Spoilers, Cross-Posted on WordPress",Book Blog Transcripts,NaN,2019-12-10,ers
1007,Dear Charlie,HeadmasterFelicis,"Dear Charlie, Dear Charlie, Dear Charlie, Dear...",http://archiveofourown.org/works/1046317,General Audiences,No Archive Warnings Apply,Gen,The Perks of Being a Wallflower - All Media Types,NaN,"Charlie (Perks of Being a Wallflower), Dear Fr...",NaN,NaN,NaN,2013-11-16,ers
1019,I Apologize,mikaylaesthetic,"Jonas was now a 17, as he had counted. Th...",http://archiveofourown.org/works/1038099,Not Rated,Choose Not To Use Archive Warnings,M/M,The Giver Series - Lois Lowry,"Sophie Devereaux/Nathan Ford, Alec Hardison/Pa...",The Giver,"Love Memories, Love, Giver, Memories",NaN,NaN,2013-11-09,ers
